# Chuẩn bị Data

In [ ]:
import os
import shutil
import pandas as pd

**Tạo hai thư mục con trong "train" là "real" và "fake", sau đó di chuyển ảnh thật vào "real" và ảnh giả vào "fake".**

In [ ]:
# Đường dẫn file CSV chứa label
csv_path = r'C:\Users\Uyen\Documents\PhanTichDLChuyenBiet\data2\label.csv'

# Đọc file CSV vào DataFrame
df = pd.read_csv(csv_path)

# Kiểm tra nếu file CSV không tồn tại hoặc rỗng
if not os.path.exists(csv_path) or os.path.getsize(csv_path) == 0:
    print("Error: CSV file is missing or empty.")
    # Thêm xử lý hoặc thoát chương trình nếu cần

else:
    df = pd.read_csv(csv_path)

    # Tạo cột mới 'name' cắt bớt 4 ký tự đuôi trong cột 'fname'
    df['name'] = df['fname'].apply(lambda x: os.path.splitext(x)[0])

    # Đường dẫn thư mục train chứa các thư mục con là hình ảnh
    train_folder = r'C:\Users\Uyen\Documents\PhanTichDLChuyenBiet\data2\train'

    # Đường dẫn thư mục real và fake
    real_folder = os.path.join(train_folder, "real")
    fake_folder = os.path.join(train_folder, "fake")

    # Tạo thư mục nếu chưa tồn tại
    os.makedirs(real_folder, exist_ok=True)
    os.makedirs(fake_folder, exist_ok=True)

    # Duyệt qua từng thư mục con trong thư mục train
    for folder in os.listdir(train_folder):
        # Lấy label từ DataFrame nếu 'name' tồn tại trong DataFrame
        if 'name' in df.columns and 'liveness_score' in df.columns:
            label = df.loc[df['name'] == folder, 'liveness_score'].values
            # Kiểm tra nếu có ít nhất một giá trị trong label
            if label.size > 0:
                label = label[0]
                # Di chuyển thư mục con vào real hoặc fake tùy thuộc vào label
                destination = real_folder if label == 1 else fake_folder
                shutil.move(os.path.join(train_folder, folder), destination)
            else:
                print(f"Warning: No label found for folder '{folder}'.")
        else:
            print("Error: 'fname' or 'liveness_score' columns not found in CSV.")

In [ ]:
# Di chuyển hình ảnh ra khỏi thư mục con trong 'real'
real_subfolders = [subfolder for subfolder in os.listdir(real_folder) if os.path.isdir(os.path.join(real_folder, subfolder))]
for subfolder in real_subfolders:
    subfolder_path = os.path.join(real_folder, subfolder)
    for file in os.listdir(subfolder_path):
        file_path = os.path.join(subfolder_path, file)
        destination_path = os.path.join(real_folder, file)

        # Check if the file already exists in the destination
        counter = 1
        while os.path.exists(destination_path):
            file_name, file_extension = os.path.splitext(file)
            destination_path = os.path.join(real_folder, f"{file_name}_{counter}{file_extension}")
            counter += 1

        shutil.move(file_path, destination_path)
    os.rmdir(subfolder_path)

# Di chuyển hình ảnh ra khỏi thư mục con trong 'fake'
fake_subfolders = [subfolder for subfolder in os.listdir(fake_folder) if os.path.isdir(os.path.join(fake_folder, subfolder))]
for subfolder in fake_subfolders:
    subfolder_path = os.path.join(fake_folder, subfolder)
    for file in os.listdir(subfolder_path):
        file_path = os.path.join(subfolder_path, file)
        destination_path = os.path.join(fake_folder, file)

        # Check if the file already exists in the destination
        counter = 1
        while os.path.exists(destination_path):
            file_name, file_extension = os.path.splitext(file)
            destination_path = os.path.join(fake_folder, f"{file_name}_{counter}{file_extension}")
            counter += 1

        shutil.move(file_path, destination_path)
    os.rmdir(subfolder_path)

print("Image files moved successfully.")

Image files moved successfully.


**Chia tập train và validation**

In [ ]:
import os
from sklearn.model_selection import train_test_split
import shutil

def split_train_validation(input_folder, output_folder, test_size=0.2, random_state=42):
    # Duyệt qua từng thư mục con trong tập train
    for class_folder in os.listdir(input_folder):
        class_path = os.path.join(input_folder, class_folder)

        # Chỉ xử lý thư mục con, bỏ qua các tệp tin
        if os.path.isdir(class_path):
            # Lấy danh sách tất cả các ảnh trong thư mục con
            images = [img for img in os.listdir(class_path) if img.endswith(".jpg")]

            # Chia tập train và validation
            train_images, val_images = train_test_split(images, test_size=test_size, random_state=random_state)

            # Tạo thư mục con trong output cho tập train và validation
            train_output_folder = os.path.join(output_folder, 'train', class_folder)
            val_output_folder = os.path.join(output_folder, 'validation', class_folder)

            os.makedirs(train_output_folder, exist_ok=True)
            os.makedirs(val_output_folder, exist_ok=True)

            # Di chuyển ảnh vào thư mục tương ứng
            for img in train_images:
                src_path = os.path.join(class_path, img)
                dest_path = os.path.join(train_output_folder, img)
                shutil.copy(src_path, dest_path)

            for img in val_images:
                src_path = os.path.join(class_path, img)
                dest_path = os.path.join(val_output_folder, img)
                shutil.copy(src_path, dest_path)

# Đường dẫn thư mục chứa tập train với hai thư mục con 'fake' và 'real'
input_train_folder = r'C:\Users\Uyen\Documents\PhanTichDLChuyenBiet\data2\train'

# Đường dẫn thư mục output cho tập train và validation
output_split_folder = r'C:\Users\Uyen\Documents\PhanTichDLChuyenBiet\data2\split'

# Tạo thư mục output nếu chưa tồn tại
os.makedirs(output_split_folder, exist_ok=True)

# Chia tập train và validation
split_train_validation(input_train_folder, output_split_folder)

print("Splitting completed successfully.")

Splitting completed successfully.
